In [1]:
#Colab specific setup
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False


Mounted at /content/drive
Note: using Google CoLab


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Titanic/test.csv')
test_ids = test["PassengerId"]
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def clean(data):
  data = data.drop(["Ticket","Cabin","Name","PassengerId"],axis=1)
  cols = ["SibSp","Parch","Fare","Age"]

  for col in cols:
    data[col].fillna(data[col].median(), inplace=True)
    data.Embarked.fillna("U", inplace=True)
    return data
data = clean(data)
test = clean(test)


In [4]:
data.tail(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
886,0,2,male,27.0,0,0,13.00,S
887,1,1,female,19.0,0,0,30.00,S
888,0,3,female,NaN,1,2,23.45,S
889,1,1,male,26.0,0,0,30.00,C
890,0,3,male,32.0,0,0,7.75,Q


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
  data[col]= le.fit_transform(data[col])
  test[col]= le.transform(test[col])
  print(le.classes_)


data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = data["Survived"]
X = data.drop("Survived", axis=1)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_imputed, y_train)


In [8]:
from sklearn.metrics import accuracy_score
X_val_imputed = imputer.transform(X_val)
predictions = clf.predict(X_val_imputed)
accuracy_score(y_val, predictions)

0.8100558659217877

In [9]:
test_imputed = imputer.transform(test)
submission_preds = clf.predict(test_imputed)

In [10]:
df = pd.DataFrame ({"PassengerId":test_ids.values,
                    "Survived":submission_preds,
})

In [11]:
path = '/content/drive/MyDrive/Titanic/'
df.to_csv(path + "Submission.csv", index=False)